# BagrutAI — Fine‑tuning an Israeli Civics (אזרחות) “Bagrut tutor”

**Final high school project**

- **Student**: *(fill in)*
- **School / Class**: *(fill in)*
- **Date**: *(fill in)*

- **Goal**: Fine‑tune a small language model to answer Israeli Civics (אזרחות) Bagrut‑style questions in **Hebrew**, with **short, factual** answers.
- **Approach**: Combine **real** Bagrut Q/A with **synthetic** Q/A generated from the study material, then fine‑tune using **LoRA**.

## What this notebook does

- **Part A — Synthetic data generation**: Generate additional Bagrut‑style questions and answers from a civics study document (`civics.docx`).
- **Part B — Dataset build**: Clean, normalize, and combine real + synthetic data into a single training dataset.
- **Part C — Fine‑tuning**: Fine‑tune `Gemma 2 2B` (4‑bit) with LoRA using Unsloth.
- **Part D — Evaluation**: Evaluate qualitatively on a **real‑only** holdout set.

## How to run (recommended order)

1. *(Optional)* Run **Part A** if you want to regenerate synthetic data.
2. Run **Part B/C** to build `master_training_data.jsonl` and fine‑tune the model.
3. Run **Part D** to compare **base vs fine‑tuned** answers on `test_real.jsonl`.

## Project files (JSONL)

- `training_data.jsonl`: **Real** Bagrut questions & answers.
- `synthetic_data_massive.jsonl`: **Synthetic** Q/A generated from the study document.
- `master_training_data.jsonl`: The **reproducible training mix** written by this notebook.
- `test_real.jsonl`: A **real‑only holdout set** for final evaluation (never used in training).

## Reproducibility

- A fixed seed is used (`SEED = 3407`) so sampling and splits are repeatable.
- The notebook writes out the exact training mix so results can be reproduced.

## Responsible use

- This model may still **make mistakes** or **hallucinate**; it should be used as a study aid.
- Real exam questions may be copyrighted; avoid redistributing proprietary materials.

## References (tools used)

- Unsloth (LoRA fine‑tuning)
- Hugging Face `datasets`
- TRL `SFTTrainer`

> Tip for your report: include screenshots of dataset sizes, training/eval loss, and before/after answers on `test_real.jsonl`.


In [ ]:
## Part A — Generate synthetic Bagrut‑style Q/A from study material

This section generates **additional practice questions** from a civics study document.

### Inputs

- **`civics.docx`**: a Word document containing the civics study material.
- **Gemini API key**: stored in Colab Secrets as `GEMINI_API_KEY`.

### Output

- **`synthetic_data_massive.jsonl`**: one JSON object per line, formatted as:

```json
{"messages": [{"role": "user", "content": "..."}, {"role": "model", "content": "..."}]}
```

### Quality guidelines (important)

- Synthetic data should be **short, factual, and exam‑style**.
- If the model returns invalid JSON, the code skips that chunk.
- After generating synthetic data, we later **mix it with real data** (and keep a real‑only test set) to reduce “AI‑style” artifacts.


In [ ]:
# 1. Install libraries
%pip install google-genai python-docx -q

from google import genai
import docx
import json
import time
from google.colab import userdata

# 2. Connect to the API
try:
    GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')
    client = genai.Client(api_key=GOOGLE_API_KEY)
    print("✅ Successfully connected to the API!")
except Exception as e:
    print("❌ Connection error. Make sure the key is defined in Secrets.")
    raise e

# 3. Read the Word file and divide into chunks
word_file_path = "/content/civics.docx" # Change this if your file name is different!

print(f"📄 Reading the file {word_file_path}...")
try:
    doc = docx.Document(word_file_path)
except Exception as e:
    raise FileNotFoundError(f"Could not find the file '{word_file_path}'. Did you upload it to the sidebar?")

# Collect all text and divide into chunks
paragraphs = [para.text for para in doc.paragraphs if para.text.strip()]
full_text = "\n".join(paragraphs)

chunk_size = 4000 # Number of characters per chunk (about 1.5 to 2 pages)
text_chunks = [full_text[i:i + chunk_size] for i in range(0, len(full_text), chunk_size)]

print(f"📚 The file was divided into {len(text_chunks)} chunks. Starting question generation!")

system_instruction = "אתה מורה עזר לאזרחות שמכין תלמידים לבגרות בישראל."
total_generated = 0

# 4. Loop through each chunk and generate questions
for index, chunk in enumerate(text_chunks):
    print(f"⏳ Processing chunk {index + 1} out of {len(text_chunks)}...")
    
    prompt = f"""
    You are an expert Israeli Civics teacher creating training data for an AI.
    Read the following text and generate up to 10 official Bagrut-style questions and answers based ONLY on this text.
    
    The answers must be short, factual, and written in Hebrew.
    
    Format your output STRICTLY as a JSON list of objects, like this:
    [
      {{"question": "...", "answer": "..."}}
    ]
    
    Text to use:
    {chunk}
    """
    
    try:
        # Call Gemini 2.5 Flash
        response = client.models.generate_content(
            model='gemini-2.5-flash',
            contents=prompt
        )
        
        # Clean and parse the JSON
        json_text = response.text.strip().replace("```json", "").replace("```", "")
        new_qa_pairs = json.loads(json_text)
        
        # Save the questions to a new file
        with open("synthetic_data_massive.jsonl", "a", encoding="utf-8") as f:
            for pair in new_qa_pairs:
                user_input = f"{system_instruction}\n\n{pair['question']}"
                entry = {
                    "messages": [
                        {"role": "user", "content": user_input},
                        {"role": "model", "content": pair['answer']}
                    ]
                }
                f.write(json.dumps(entry, ensure_ascii=False) + "\n")
        
        total_generated += len(new_qa_pairs)
        print(f"✅ Added {len(new_qa_pairs)} questions (Total so far: {total_generated}).")
        
        # A short 3-second pause to respect the API quota (Rate Limit)
        time.sleep(3)
        
    except Exception as e:
        print(f"⚠️ Error in chunk {index + 1}, the model probably returned an invalid format. Skipping to the next chunk.")

print(f"🎉 We are done! The file 'synthetic_data_massive.jsonl' is ready with {total_generated} questions and answers.")

## Part B/C — Build dataset + Fine‑tune with LoRA (Unsloth)

This section:

1. Loads **real** (`training_data.jsonl`) and **synthetic** (`synthetic_data_massive.jsonl`) datasets.
2. Cleans and normalizes message format (including fixing synthetic rows where the “system” prompt was embedded inside the user message).
3. Creates a **real‑only holdout test set** (`test_real.jsonl`).
4. Builds a controlled **train mix** (default: **80% real / 20% synthetic**) and saves it as `master_training_data.jsonl`.
5. Fine‑tunes a small instruction model using **4‑bit + LoRA**, which is practical on Google Colab GPUs.

### Why Gemma 2 2B + LoRA?

- **Small enough** to train on Colab.
- **LoRA** updates only a small number of parameters, making training fast and memory‑efficient.
- **4‑bit** quantization reduces VRAM usage while keeping good quality for an educational demo.

### Key outputs

- `outputs/lora_adapter/`: the trained LoRA adapter (can be reloaded later).
- `test_real.jsonl`: the real‑only holdout set used for final evaluation.


In [ ]:
# 1. INSTALL & SETUP
try:
    import unsloth
except ImportError:
    print("⏳ Installing Unsloth... (approx 2 mins)")
    %pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
    %pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

import torch
from unsloth import FastLanguageModel
from datasets import Dataset, load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
import json
import os
import random

# 2. LOAD MODEL (Switched to 2B for stability)
# The 2B model is lighter and runs perfectly on Colab's standard GPU
max_seq_length = 2048
dtype = None
load_in_4bit = True

print("⏳ Loading Gemma 2 (2B) model...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-2b-it-bnb-4bit",  # <--- CHANGED TO 2B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# 3. PREPARE DATA
print("⏳ Preparing dataset...")

REAL_FILE = "training_data.jsonl"
SYNTH_FILE = "synthetic_data_massive.jsonl"
MASTER_FILE = "master_training_data.jsonl"  # optional fallback

SYNTH_RATIO = 0.20           # target fraction of synthetic in training mix
REAL_TEST_FRACTION = 0.10    # real-only holdout fraction
SEED = 3407

teacher_prefix = "אתה מורה עזר לאזרחות"


def read_jsonl(path):
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            rows.append(json.loads(line))
    return rows


def normalize_messages(messages):
    if not isinstance(messages, list):
        return None

    sys = next((m.get("content") for m in messages if isinstance(m, dict) and m.get("role") == "system"), "")
    usr = next((m.get("content") for m in messages if isinstance(m, dict) and m.get("role") == "user"), "")
    mod = next(
        (m.get("content") for m in messages if isinstance(m, dict) and m.get("role") in {"model", "assistant"}),
        "",
    )

    # Synthetic file often embedded the system instruction in the user message.
    if (not sys) and isinstance(usr, str):
        u = usr.strip()
        if u.startswith(teacher_prefix) and "\n\n" in u:
            sys_candidate, rest = u.split("\n\n", 1)
            if sys_candidate.strip().startswith(teacher_prefix):
                sys = sys_candidate.strip()
                usr = rest.strip()

    if not isinstance(usr, str) or not usr.strip() or not isinstance(mod, str) or not mod.strip():
        return None

    norm = []
    if isinstance(sys, str) and sys.strip():
        norm.append({"role": "system", "content": sys.strip()})
    norm.append({"role": "user", "content": usr.strip()})
    norm.append({"role": "model", "content": mod.strip()})
    return norm


def build_dataset_from_sources():
    has_real = os.path.exists(REAL_FILE)
    has_synth = os.path.exists(SYNTH_FILE)

    if has_real and has_synth:
        real_rows = read_jsonl(REAL_FILE)
        synth_rows = read_jsonl(SYNTH_FILE)

        real = []
        synth = []

        for r in real_rows:
            msgs = normalize_messages(r.get("messages"))
            if msgs is not None:
                real.append({"messages": msgs, "source": "real"})

        for r in synth_rows:
            msgs = normalize_messages(r.get("messages"))
            if msgs is not None:
                synth.append({"messages": msgs, "source": "synthetic"})

        if not real:
            raise ValueError("Real dataset loaded but produced 0 usable rows after cleaning.")

        rng = random.Random(SEED)

        # Real-only test set
        n_test = max(10, int(len(real) * REAL_TEST_FRACTION))
        n_test = min(n_test, max(10, len(real) - 1))
        test_real = rng.sample(real, n_test)
        test_ids = {id(x) for x in test_real}
        real_train = [x for x in real if id(x) not in test_ids]

        # Keep all real_train; downsample synthetic to desired ratio
        if synth and 0.0 < SYNTH_RATIO < 1.0:
            target_synth = int(len(real_train) * (SYNTH_RATIO / (1.0 - SYNTH_RATIO)))
            target_synth = max(0, min(target_synth, len(synth)))
            synth_selected = rng.sample(synth, target_synth) if target_synth else []
        else:
            synth_selected = []

        combined = real_train + synth_selected
        rng.shuffle(combined)

        # Save the real-only test set for final evaluation/demo
        with open("test_real.jsonl", "w", encoding="utf-8") as f:
            for row in test_real:
                f.write(json.dumps(row, ensure_ascii=False) + "\n")

        # Save combined training set for reproducibility
        with open(MASTER_FILE, "w", encoding="utf-8") as f:
            for row in combined:
                f.write(json.dumps(row, ensure_ascii=False) + "\n")

        print(
            "✅ Built combined dataset from sources:\n"
            f"- real usable: {len(real)}\n"
            f"- synthetic usable: {len(synth)}\n"
            f"- train mix (real): {len(real_train)}\n"
            f"- train mix (synthetic): {len(synth_selected)}\n"
            f"- saved: {MASTER_FILE} (train mix)\n"
            f"- saved: test_real.jsonl (real-only holdout)"
        )

        return Dataset.from_list(combined)

    # Fallback to an already-combined file
    for p in [MASTER_FILE, REAL_FILE]:
        if os.path.exists(p):
            print(f"✅ Using existing dataset file: {p}")
            return load_dataset("json", data_files=p, split="train")

    raise FileNotFoundError(
        f"Could not find '{REAL_FILE}' and '{SYNTH_FILE}', and no fallback '{MASTER_FILE}'."
    )


def format_chat_template_fixed(examples):
    texts = []
    for messages in examples["messages"]:
        if not isinstance(messages, list):
            texts.append("")
            continue

        sys = next((m.get("content") for m in messages if isinstance(m, dict) and m.get("role") == "system"), "")
        usr = next((m.get("content") for m in messages if isinstance(m, dict) and m.get("role") == "user"), "")
        mod = next((m.get("content") for m in messages if isinstance(m, dict) and m.get("role") == "model"), "")

        user_input = f"{sys}\n\n{usr}" if sys else usr
        if not isinstance(user_input, str) or not user_input.strip() or not isinstance(mod, str) or not mod.strip():
            texts.append("")
            continue

        conversation = tokenizer.apply_chat_template(
            [
                {"role": "user", "content": user_input.strip()},
                {"role": "model", "content": mod.strip()},
            ],
            tokenize=False,
            add_generation_prompt=False,
        )
        texts.append(conversation)

    return {"text": texts}

raw_dataset = build_dataset_from_sources()
dataset = raw_dataset.map(format_chat_template_fixed, batched=True)
dataset = dataset.filter(lambda x: bool(x.get("text")))

split_ds = dataset.train_test_split(test_size=0.05, seed=SEED)
train_dataset = split_ds["train"]
eval_dataset = split_ds["test"]

# 4. SETUP LORA
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

# 5. TRAIN
print("🚀 Starting Training... (This will be fast!)")

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        evaluation_strategy = "steps",
        eval_steps = 10,
        save_strategy = "steps",
        save_steps = 10,
        save_total_limit = 2,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

trainer.train()

trainer.model.save_pretrained("outputs/lora_adapter")
tokenizer.save_pretrained("outputs/lora_adapter")

print("🎉 Training Complete! Saved adapter to outputs/lora_adapter")

## Part D — Evaluation (real‑only holdout)

To make this a strong final project, we evaluate on a **real‑only** set (`test_real.jsonl`) that was **never used for training**.

### What to look for

- **Correctness**: does the answer match the civics material?
- **Bagrut style**: short, factual, uses proper terms (e.g., עקרונות דמוקרטיים, חוקי יסוד).
- **No hallucinations**: if unsure, the answer should be conservative.

### Suggested write‑up structure (easy to grade)

- **Data**: where each dataset came from (real vs synthetic), and why mixing helps.
- **Method**: LoRA fine‑tuning (why it’s efficient), model choice, key hyperparameters.
- **Evaluation**: real‑only holdout + before/after examples.
- **Limitations**: dataset size, synthetic artifacts, no guarantee of correctness.
- **Future work**: more real questions, dedup, more question types (קטע/אירוע), better refusal behavior.


In [ ]:
# Demo: compare base vs fine-tuned on real-only holdout
# Run this AFTER training (requires outputs/lora_adapter and test_real.jsonl).

import json
import os
import random
import torch

from peft import PeftModel
from unsloth import FastLanguageModel

BASE_MODEL_NAME = "unsloth/gemma-2-2b-it-bnb-4bit"
ADAPTER_DIR = "outputs/lora_adapter"
HOLDOUT_FILE = "test_real.jsonl"

N_SAMPLES = 10
MAX_NEW_TOKENS = 200
SEED = 3407


def load_holdout(path):
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            rows.append(json.loads(line))
    return rows


def extract_sys_user(messages):
    sys = ""
    usr = ""
    for m in messages:
        if not isinstance(m, dict):
            continue
        if m.get("role") == "system" and not sys:
            sys = m.get("content", "")
        if m.get("role") == "user" and not usr:
            usr = m.get("content", "")
    return (sys or "").strip(), (usr or "").strip()


def generate_answer(model, tokenizer, sys, usr):
    user_input = f"{sys}\n\n{usr}" if sys else usr
    prompt = tokenizer.apply_chat_template(
        [{"role": "user", "content": user_input}],
        tokenize=False,
        add_generation_prompt=True,
    )

    device = "cuda" if torch.cuda.is_available() else "cpu"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS,
            do_sample=False,
        )

    answer_tokens = out[0][inputs["input_ids"].shape[-1] :]
    return tokenizer.decode(answer_tokens, skip_special_tokens=True).strip()


# Load holdout questions
if not torch.cuda.is_available():
    print("⚠️ GPU not detected. This will be slow on CPU.")

if not (os.path.exists(HOLDOUT_FILE) and os.path.exists(ADAPTER_DIR)):
    raise FileNotFoundError(
        f"Missing '{HOLDOUT_FILE}' or '{ADAPTER_DIR}'. Run the training cell first."
    )

rows = load_holdout(HOLDOUT_FILE)
rng = random.Random(SEED)
rng.shuffle(rows)
rows = rows[: min(N_SAMPLES, len(rows))]

# Load base model
base_model, base_tokenizer = FastLanguageModel.from_pretrained(
    model_name=BASE_MODEL_NAME,
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)
base_model = FastLanguageModel.for_inference(base_model)

print("\n===== BASE MODEL (before fine-tuning) =====\n")
base_answers = []
for i, r in enumerate(rows, 1):
    sys, usr = extract_sys_user(r.get("messages", []))
    ans = generate_answer(base_model, base_tokenizer, sys, usr)
    base_answers.append(ans)
    print(f"[{i}] שאלה: {usr}\nתשובה (בסיס): {ans}\n")

# Load LoRA adapter on top of the base model
ft_model = PeftModel.from_pretrained(base_model, ADAPTER_DIR)
ft_model = FastLanguageModel.for_inference(ft_model)

print("\n===== FINE-TUNED MODEL (LoRA adapter) =====\n")
for i, r in enumerate(rows, 1):
    sys, usr = extract_sys_user(r.get("messages", []))
    ans = generate_answer(ft_model, base_tokenizer, sys, usr)
    print(f"[{i}] שאלה: {usr}\nתשובה (מאומן): {ans}\n")

print("Done.")
